# Multi-Agentic Financial Analyst
This notebook will be the test build of the multi-agentic AI system.

In [1]:
%package install boto3=1.34.51 langchain 

Running: micromamba install boto3=1.34.51 langchain --yes --log-level=error

  Package        Version  Build             Channel         Size
──────────────────────────────────────────────────────────────────
  Upgrade:
──────────────────────────────────────────────────────────────────

  - botocore     1.29.37  pyhd8ed1ab_0      conda-forge      6MB
  + botocore    1.34.162  pyge38_1234567_0  conda-forge      7MB
  - s3transfer     0.6.0  pyhd8ed1ab_0      conda-forge     57kB
  + s3transfer    0.10.4  pyhd8ed1ab_1      conda-forge     63kB
  - boto3        1.26.37  pyhd8ed1ab_0      conda-forge     76kB
  + boto3        1.34.51  pyhd8ed1ab_0      conda-forge     81kB

  Summary:

  Upgrade: 3 packages

  Total download: 7MB

Note: Packages not from Bloomberg channels are not vetted by Bloomberg.
Please restart the Jupyter kernel if you run into any issues after installing or updating packages via %package.



In [2]:
!pip -q install langchain_aws langchain-huggingface

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.17 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 0.3.54 which is incompatible.
langchain-text-splitters 0.2.4 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.54 which is incompatible.
langchain-community 0.2.19 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 0.3.54 which is incompatible.


In [3]:
import boto3
import botocore
import json
import importlib

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.rate_limiters import InMemoryRateLimiter

from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field
from IPython.display import Markdown, display

import prompts


import concurrent.futures

In [4]:
importlib.reload(prompts)

<module 'prompts' from '/project/prompts.py'>

In [5]:
# Bedrock client initialise
config = botocore.config.Config(read_timeout=1000)
boto3_bedrock = boto3.client('bedrock-runtime',config=config)

with open('Data/prompts.json', 'rb') as f:
    prompt_set = json.load(f)

### Configure the model

In [6]:
model_claude_id = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
model_llama_id = 'us.meta.llama3-1-70b-instruct-v1:0'

# set up the LLM in Bedrock
rate_limiter = InMemoryRateLimiter(
    requests_per_second=50,
    check_every_n_seconds=1,
    max_bucket_size=10,
)

llm = ChatBedrock(
    client = boto3_bedrock,
    model = model_claude_id,
    temperature = 0.01,
    max_tokens=4000,
    rate_limiter = rate_limiter
)

In [7]:
system_prompt = prompts.SYSTEM_PROMPTS['COT_CLAUDE_TEST']['prompt']
prompt_template = PromptTemplate.from_template(system_prompt)
# Create the prompt to feed into the model
prompt_in = prompt_template.format(financials=prompt_set[0]['prompt'][1]['content'])

In [9]:
output = llm.invoke(prompt_in)
    # decision_dict = {
    #     'date': prompt['date'],
    #     'security': prompt['security'],
    #     'response': output
    # }
    # return decision_dict

In [10]:

display(Markdown(output.content))

# Financial Analysis Report: Earnings Forecast

## 1. Analysis of Current Profitability, Liquidity, Solvency, and Efficiency Ratios

### Profitability Ratios
- **Gross Profit Margin**: 21.24% (t) vs 21.61% (t-1)
- **Operating Profit Margin**: 3.06% (t) vs 3.25% (t-1)
- **Net Profit Margin**: 2.52% (t) vs 2.69% (t-1)
- **Return on Assets (ROA)**: 3.88% (t) vs 4.08% (t-1)
- **Return on Equity (ROE)**: 14.35% (t) vs 15.23% (t-1)

### Liquidity Ratios
- **Current Ratio**: 0.66 (t) vs 0.66 (t-1)
- **Quick Ratio**: 0.32 (t) vs 0.31 (t-1)
- **Cash Ratio**: 0.03 (t) vs 0.03 (t-1)

### Solvency Ratios
- **Debt-to-Equity Ratio**: 1.73 (t) vs 1.73 (t-1)
- **Debt-to-Assets Ratio**: 0.47 (t) vs 0.46 (t-1)
- **Interest Coverage Ratio**: 6.15 (t) vs 6.31 (t-1)

### Efficiency Ratios
- **Asset Turnover**: 1.54 (t) vs 1.51 (t-1)
- **Inventory Turnover**: 11.32 (t) vs 10.22 (t-1)
- **Receivables Turnover**: 18.32 (t) vs 18.48 (t-1)

## 2. Time-Series Analysis Across Ratios

### Profitability Trends
- **Gross Profit Margin**: Declining trend from 23.10% (t-5) to 21.24% (t)
- **Operating Profit Margin**: Consistent decline from 4.57% (t-5) to 3.06% (t)
- **Net Profit Margin**: Steady decline from 3.96% (t-5) to 2.52% (t)
- **ROE**: Decreasing from 20.28% (t-5) to 14.35% (t)

### Liquidity Trends
- **Current Ratio**: Relatively stable but consistently below 1.0, indicating potential short-term liquidity challenges
- **Quick Ratio**: Stable around 0.3-0.32, showing consistent reliance on inventory

### Solvency Trends
- **Debt-to-Equity**: Increased from 0.44 (t-5) to 1.73 (t), indicating significantly higher leverage
- **Interest Coverage Ratio**: Declined from 9.79 (t-5) to 6.15 (t), showing reduced ability to service debt

### Efficiency Trends
- **Asset Turnover**: Improved slightly from 1.92 (t-5) to 1.54 (t)
- **Inventory Turnover**: Improved from 9.43 (t-5) to 11.32 (t), indicating better inventory management

## 3. Analysis of Financial Performance

The company shows concerning trends in its financial performance:

1. **Declining Profitability**: All profitability metrics show a consistent downward trend over the six periods. Operating income has decreased by 31% from t-5 to t (from $6.15B to $4.24B).

2. **Revenue Stagnation**: Despite a slight increase in revenue (3.1% growth from t-5 to t), the company has been unable to translate this into profit growth due to rising costs.

3. **Cost Pressure**: Cost of revenue has increased at a faster rate than revenue (5.6% vs 3.1% from t-5 to t), squeezing margins.

4. **Debt Structure Change**: The company has significantly increased its long-term debt, particularly with operating leases now being capitalized, which explains the jump in LT debt between t-2 and t-1.

5. **Cash Flow Concerns**: Cash and equivalents have decreased by 19.2% from t-5 to t, suggesting potential cash flow challenges.

## 4. Stock Price Analysis

The stock price shows a concerning downward trend:
- Peak price of $59.27 in December 2019
- Significant decline to $40.32 by April 2020, representing a 32% drop from peak
- The stock price decline aligns with the deteriorating financial performance
- The market appears to be pricing in the company's declining profitability and operational challenges

## 5. Decision Analysis

### Positive Factors (30% weight)
1. **Revenue Growth**: Slight but consistent revenue growth (0.94% from t-1 to t)
2. **Improved Inventory Management**: Higher inventory turnover ratio indicates better operational efficiency
3. **Stable Asset Base**: Total assets have remained relatively stable

### Negative Factors (70% weight)
1. **Declining Profitability**: Consistent decrease in all profitability metrics
   - Operating income decreased by 5.1% from t-1 to t
   - Net income decreased by 5.7% from t-1 to t
2. **Margin Compression**: Gross profit margin declined from 21.61% to 21.24%
3. **Increasing Cost Structure**: Cost of revenue growing faster than revenue
4. **High Leverage**: Debt-to-equity ratio remains high at 1.73
5. **Declining EPS**: Basic EPS decreased from $4.08 to $3.90 (4.4% decline)
6. **Stock Price Decline**: Significant market devaluation reflecting negative sentiment

## 6. Final Decision

Based on the comprehensive analysis of financial statements, I predict that earnings will **decrease** in the next financial period by approximately 5-8%. The company shows consistent deterioration in profitability metrics, margin compression, and increasing cost pressures without corresponding revenue growth to offset these challenges.

**Confidence Score: 75%**

The primary drivers for this decision are the consistent multi-year downward trend in operating income, net profit margin, and EPS, combined with rising costs and limited revenue growth. The market sentiment, as reflected in the declining stock price, further supports this negative outlook.

## 7. Information Wished for Better Decision-Making

Additional information that would enhance the analysis includes:

1. **Segment reporting** to identify which business lines are underperforming and which might be growth drivers. This would help assess if declining profitability is company-wide or concentrated in specific areas.

2. **Forward-looking guidance** from management regarding cost-cutting initiatives, strategic investments, or market expansion plans that could reverse the negative trends.

3. **Industry benchmarking data** to determine if the company's challenges are company-specific or industry-wide, which would help assess competitive positioning.

4. **Cash flow statement details** to better understand operational cash generation, capital expenditures, and financing activities, which would provide insight into sustainability of operations and dividend payments.

5. **Customer concentration metrics** to evaluate dependency risks and potential revenue stability.

These additional data points would provide context for the declining performance and help assess whether the company has viable strategies to reverse the negative trends.